In [1]:
#import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_regression
import pickle
import hashlib


In [5]:
#Load and print the data

#Enter the answer here
data = pd.read_csv("student-mat.csv")
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [6]:
data.isnull().values.any()
data.fillna(data.mean(),inplace=True)
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


# Exploratory Data Analysis

Question:1

Which school has more number of students? Gabriel Pereira or Mousinho da Silveira

Assign the number of students in that school to the variable q1

In [7]:
#Enter the answer here
school_max_students=data['school'].value_counts().idxmax()
q1 = data['school'].value_counts().max()
print(school_max_students, q1)

GP 349


Analysis: The GP (Gabriel Pereira) school has maximum number of students

-------------------------------------------------------------------
Question 2:

Which group of students have more opportunity in schooling? Rural or Urban?

Assign the number of rural students or the number of urban students who has more opportunity to q2

In [8]:
#Enter the answer here
address_max_students=data['address'].value_counts().idxmax()
q2 = data['address'].value_counts().max()
print(address_max_students, q2)

U 307


Analysis: The Urban Students have more opportunity in schooling as compared to rural students

---------------------------------------------------------------------------------------------------------------------------
Question 3

Which group of students(rural or urban) shows more interest in each of the schools?

Instructions

Find the number of rural and Urban areas students in each school and find the greatest among them.
Store them in respective variable

In [11]:
df = data[['school','address']]
df = df.groupby(['school','address']).size()
print(df)

school  address
GP      R           63
        U          286
MS      R           25
        U           21
dtype: int64


Above Stats depict the groupng of students count based on school and address

In [12]:
#Store the number of rural students or the number of urban students who had more opportunity to study in GP school
#Enter the answer here
q3 = df.loc['GP'].max()
print("GP U : ", q3)

#Store the number of rural students or the number of urban students who had more opportunity to study in GP school
#Enter the answer here
q4=df.loc['MS'].max()
print("MS R : ", q4)

GP U :  286
MS R :  25


Analysis: The Urben studens are more incline towards the GP scholl while the rural students are more inclined towards the MS school

------------------------------------------------------------------------------------------------------------

Question 4

Which age group of students are more interested in GP school? Assign the specific age to the variable.

In [14]:
df1 = data[['school','age']]
df1 = df1.groupby(['school','age']).size()
print(df1)

#Enter the answer here
q5= df1.loc['GP'].idxmax()
print("GP : ", q5)

school  age
GP      15      82
        16     104
        17      86
        18      57
        19      18
        20       1
        22       1
MS      17      12
        18      25
        19       6
        20       2
        21       1
dtype: int64
GP :  16


the above status depict the student inclination to each school based on the age group

-----------------------------------------------------------------------------------------------------------------------------

Question 5

Find out the average Daily consumption of alcohol in rural area and Urban area separately.

In [17]:
df3 = data[['address','Dalc']].groupby(['address'])['Dalc'].mean()
print(df3)

#Average Daily Consumption of Alcohol in Rural Area

#Enter the answer here
q6= df3.loc['R'].mean()


#Average Daily Consumption of Alcohol in Urban Area

#Enter the answer here
q7= df3.loc['U'].mean()


address
R    1.636364
U    1.436482
Name: Dalc, dtype: float64


Analysis: The Daily Average consumption of alcohol is more in rural areas as compared to urban areas

---------------------------------------------------------------------------------------------------------------------------
Question 6

Find out the average Daily consumption of alcohol in rural areas and Urban areas separately specific to each school and assign the greater average value to the respective variable of schools.

In [19]:
dfGP = data[data['school'] == 'GP'].groupby('address')['Dalc'].mean()
print(dfGP)

address
R    1.492063
U    1.433566
Name: Dalc, dtype: float64


The above stats show the daily average alcohol consumption for GP school students categories based on rural and urban

In [20]:
dfMS = data[data['school'] == 'MS'].groupby('address')['Dalc'].mean()
print(dfMS)

address
R    2.00000
U    1.47619
Name: Dalc, dtype: float64


The above stats show the daily average alcohol consumption for MS school students categories based on rural and urban

-------------------------------------------------------------------------------------------------------------------------------
Question 7

Let us find the variable which made the maximum impact on the Final grade of students with pearson coefficient.

Note: All the variables used for calculating pearson coefficient must be numerical.

A predictor that has a low p-value is likely to be a meaningful addition to your model because changes in the predictor's value are related to changes in the response variable.

Conversely, a larger (insignificant) p-value suggests that changes in the predictor are not associated with changes in the response.

Instructions

Consider G3 as target variable Y.

Remove the variables G1, G2. Consider the rest as predictor variables

Calculate the P value of all variables (hint: fregression and label encoding using cat.codes)

Variables with P-values less than 0.05 (95% confidence) are considered to be the Predictor Variables. Form a dataframe X with it.

Calculate the correlation between the predictor variable and target variable

Find the highest Correlation value in the column G3 which depicts the variable that made the greatest impact on G3.

Print the variable that made greatest impact on G3.

Assign the highest correlation value and the highest impacted attribute to the respective variable

Note: Make sure to compare the absolute value of the correlation values

In [22]:
Y = data['G3']
data = data.drop(['G3','G2','G1'], axis =1)
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,4
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,2
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,4


Dataset after dropping G1, G2, G3 columns and Taget variable Y assigned with G3

In [24]:
from sklearn.feature_selection import f_regression

j=0
a = data.dtypes
for i in a :
    if i == 'object':
        h = data.columns[j]
    data[h]= data[h].astype('category').cat.codes
    j+=1
(f,p_value) = f_regression(data,Y)
p_value = pd.DataFrame({'p_value':p_value})
X = data[['school','age','address','famsize','Medu','Fedu','Mjob','reason','guardian','traveltime','studytime','failures','schoolsup','higher','internet','romantic','freetime','goout','Dalc','Walc','health']]
corv = X.corrwith(Y,axis=0)

In [25]:
#Highest impact attribute

#Enter the answer here
q10=abs(corv).idxmax()
print(q10)

#highest correlation value

#Enter the answer here
q11=abs(corv).max()
print(q11)

failures
0.3604149404616497


The failures make greater impact on students final grades G3

The highest coerection belween the impact variable with the target variable is 0.36